# LLM Testbench Generation Training on Colab\n\nThis notebook trains the model on Google Colab with GPU acceleration.

## 1. Setup Environment

In [ ]:
# Check GPU availability\n!nvidia-smi\n\nimport torch\nprint(f\"PyTorch version: {torch.__version__}\")\nprint(f\"CUDA available: {torch.cuda.is_available()}\")\nif torch.cuda.is_available():\n    print(f\"GPU: {torch.cuda.get_device_name(0)}\")

## 2. Mount Google Drive (Optional)

In [ ]:
# Mount Google Drive to save/load files\nfrom google.colab import drive\ndrive.mount('/content/drive')

## 3. Upload and Extract Project Files

In [ ]:
# Option 1: Upload from local computer\nfrom google.colab import files\nuploaded = files.upload()  # Select llm_testbench_colab.zip\n\n# Extract\n!unzip -q llm_testbench_colab.zip\n!ls -la

In [ ]:
# Option 2: Copy from Google Drive (uncomment if using)\n# !cp /content/drive/MyDrive/llm_testbench_colab.zip .\n# !unzip -q llm_testbench_colab.zip

## 4. Install Dependencies

In [ ]:
# Install requirements\n!pip install -q -r requirements_colab.txt\n\n# Verify installations\n!pip list | grep -E \"torch|transformers|peft|bitsandbytes\"

## 5. Configure Weights & Biases (Optional)

In [ ]:
# Set up wandb for experiment tracking\nimport os\n\n# Option 1: Set API key directly\n# os.environ['WANDB_API_KEY'] = 'your_api_key_here'\n\n# Option 2: Login interactively\n# import wandb\n# wandb.login()

## 6. Update Config for Colab GPU

In [ ]:
# Create optimized config for Colab\nimport yaml\n\n# Load existing config\nwith open('configs/config.yaml', 'r') as f:\n    config = yaml.safe_load(f)\n\n# Update for GPU training\nconfig['model']['quantization']['load_in_4bit'] = True  # Enable 4-bit quantization\nconfig['training']['per_device_train_batch_size'] = 4  # Increase batch size\nconfig['training']['per_device_eval_batch_size'] = 4\nconfig['training']['gradient_accumulation_steps'] = 4\nconfig['training']['fp16'] = True  # Enable mixed precision\nconfig['training']['num_train_epochs'] = 5  # More epochs\n\n# Optionally use larger model if you have enough GPU memory\n# config['model']['base_model'] = \"meta-llama/Llama-2-7b-hf\"  # Requires HF token\n\n# Save updated config\nwith open('configs/config_colab.yaml', 'w') as f:\n    yaml.dump(config, f)\n\nprint(\"Config updated for Colab GPU training\")

## 7. Run Training

In [ ]:
# Set config file\n!cp configs/config_colab.yaml configs/config.yaml\n\n# Run training\n!python scripts/train.py

## 8. Save Trained Model

In [ ]:
# Compress the trained model\n!cd models/checkpoints && zip -r /content/trained_model.zip * && cd /content\n!ls -lh trained_model.zip

In [ ]:
# Option 1: Download to local computer\nfrom google.colab import files\nfiles.download('trained_model.zip')

In [ ]:
# Option 2: Save to Google Drive\n!cp trained_model.zip /content/drive/MyDrive/\nprint(\"Model saved to Google Drive\")

## 9. Test Generation (Optional)

In [ ]:
# Quick test of the trained model\nfrom transformers import AutoModelForCausalLM, AutoTokenizer\nfrom peft import PeftModel\n\n# Load model\nbase_model_name = config['model']['base_model']\nmodel_path = \"./models/checkpoints\"\n\ntokenizer = AutoTokenizer.from_pretrained(model_path)\nmodel = AutoModelForCausalLM.from_pretrained(\n    base_model_name,\n    torch_dtype=torch.float16,\n    device_map=\"auto\"\n)\nmodel = PeftModel.from_pretrained(model, model_path)\n\n# Test generation\ntest_prompt = \"\"\"Generate a Verilog testbench for the following design under test (DUT):\n```verilog\nmodule counter(\n    input clk,\n    input reset,\n    output reg [3:0] count\n);\n    always @(posedge clk or posedge reset) begin\n        if (reset)\n            count <= 0;\n        else\n            count <= count + 1;\n    end\nendmodule\n```\n\n### Response:\"\"\"\n\ninputs = tokenizer(test_prompt, return_tensors=\"pt\").to(model.device)\noutputs = model.generate(**inputs, max_new_tokens=500, temperature=0.7)\nprint(tokenizer.decode(outputs[0], skip_special_tokens=True))